## Building the classifier

This notebook builds a CNN model based on our custom open sandwiches dataset. It is meant to be self-contained, runnable end-to-end, and to save the CNN to disk at a specific location at the end of the notebook - more on that later.

In [ ]:
import t4

In [ ]:
!mkdir ../data/ 2>/dev/null
!mkdir ../data/images_cropped/ 2>/dev/null

We start off by localizing the dataset that we are working with. If you are running this notebook locally and have already localized the data (e.g. you've used `openimager` to download them again yourself) you can skip this line.

In [ ]:
t4.Package.install(
    'quilt/open_images', 
    registry='s3://alpha-quilt-storage', 
    dest='../data/images_cropped/'
)

In [ ]:
# remove the hot dog class if present to make this a binary classification problem
# this is an artifact from an earlier version of this demo which was three-class instead of two-class
!rm -rf ../data/images_cropped/quilt/open_images/Hot_dog 2>/dev/null

The neural network definition and training code follows.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1/255
)

train_generator = train_datagen.flow_from_directory(
    '../data/images_cropped/quilt/open_images/',
    target_size=(128, 128),
    batch_size=16,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    '../data/images_cropped/quilt/open_images/',
    target_size=(128, 128),
    batch_size=16,
    class_mode='binary'
)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.losses import binary_crossentropy
from keras.callbacks import EarlyStopping
from keras.optimizers import RMSprop


model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(128, 128, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss=binary_crossentropy,
              optimizer=RMSprop(lr=0.0005),  # half of the default lr
              metrics=['accuracy'])

In [ ]:
import pathlib

sample_size = len(list(pathlib.Path('../data/images_cropped/').rglob('./*')))
batch_size = 16

In [ ]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=sample_size // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=round(sample_size * 0.2) // batch_size,
    callbacks=[EarlyStopping(monitor='val_loss', min_delta=0, patience=4)]
)

Finally once we've fitted the model it's time to save it to disk. We will save the model artifact to a very particular location on disk: `/opt/ml/model/clf.h5`.

This is done so that this notebook is compatible with AWS SageMaker. I personally ran this notebook training session using a Quilt AWS account and the [alekseylearn](https://github.com/ResidentMario/alekseylearn) library (an alpha-level side project) using the following CLI command:

```bash
alekseylearn fit build.ipynb --config.output_path="s3://alpha-quilt-storage/aleksey/sagemaker/alekseylearn" --config.role_name="aleksey_sagemaker_role"
```

However it is much easier and less yak-shavy to just run this notebook locally and save the model someplace on your local disk instead. To do that change the savefile locations in the following code line to something more convenient.

In [ ]:
# if running locally change this to a convenient local path, e.g. './clf.h5'
model.save("/opt/ml/model/clf.h5")

import json
with open("/opt/ml/model/history.json", "w") as f:
    # this also
    json.dump(hist.history, f)